# Decision Trees
- Decision Trees are considered one of the most mature, traditional, algorithms in predictive analytics
- They are typically used to solve classification problems through visual and explicit representations of decisions and decision making.
- Think of them like a map where you follow each path according to your decision, and each path leads to a new choice to make until you reach the end.
- They mimic the way you probably make decisions in your daily life:

### Terminology 
- **Root:** Our starting point for the tree. Note that a decision tree is drawn upside down since its root is at the top
- **Branch:** Also known as an edge, these lead from condition to condition, down to the results
- **Condition:** Also known as an internal node, this is the choice that needs to be made in order to figure out which branch to take.
- **Leaf:** Also known as a decision, these are the final results that signify the classification of the data. There are no branches coming out of a leaf, only going in to it.


### Why and when do we need Decision Trees?¶
- When features are Categorical
    - When we can classify data into known groups
- When we want to model a set of sequential, hierarchical decisions that lead to some final result. This result is the known group that the data point would be categorized into
- When we need to explain the reason for a particular decision

#### Example use cases:
- Sales and marketing departments might need a complete description of rules that influence the acquisition of a customer before they start their campaign activities
- Product planning (do we build this product or not?)
- Determining someone is a good or bad level of risk

### What do we use to obtain the root node?
- Conditional Probability
- Entropy
- Information Gain

## Decision Trees are based on Entropy
### Activity: Calculate the entropy for a coin
- Entropy shows the uncertainy of a random variable. The higher the entropy value, the more unncertain we are. Entropy is displayed as $H(X)$, where $X$ is a random variable
- The Entropy formula is the summation of probabilities multiplied by the log of probabilities:

### Entropy of coin

$H(Coin) = \sum -p(outcome) * log_2(p(outcome)$
Entropy of a fair coin
for p(outcome) in [p(H)=0.5, p(T)=0.5]):

$H(Coin) = \sum -p(outcome) * log_2(p(outcome)$

Do the following in pairs:
- Create a function entropy that takes an array of probabilities as input, and returns the entropy using the formula above
    - numpy's array, log2, and sum functions should be useful here
- Show that the fair coin has the largest entropy (uncertainty) by trying different values for the probability of heads and tails
    - i.e. show that a fair coin [.5, .5] has a larger entropy than a coin with [.9, .1] probabilities

In [1]:
import numpy as np

def entropy(prob_arr):
    H = np.array([-i*np.log2(i) for i in prob_arr])
    return H

print(entropy([.5, .5]))
print(entropy([.9, .1]))

[0.5 0.5]
[0.13680278 0.33219281]


## Let's build a Decision Tree for the Tennis Data

### Activity: Obtain the following quantitites:
- entropy for outcome
- entropy for outcome based on weak wind
- entropy for outcome based on strong wind

In [2]:
import pandas as pd

tennis = pd.read_csv('../Data/Tennis.csv')
tennis

,Day,Outlook,Temperature,Humidity,Wind,Decision
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes
5,6,Rain,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rain,Mild,Normal,Weak,Yes


In [3]:
# Entropy for Outcome
prob_arr = [9/14, 5/14]
print('Entropy of Outcome')
entropy_outcome = entropy(prob_arr)
print(entropy_outcome)
print(entropy_outcome.sum())

Entropy of Outcome
[0.40977638 0.53050958]
0.9402859586706311


In [4]:
# Entroby for Outcome based on weak wind
print('Entropy of Outcome based on weak wind')
weak_decision = dict(tennis[tennis['Wind'] == 'Weak']['Decision'].value_counts())
print(weak_decision)
entropy_weak = entropy([weak_decision['Yes']/8, weak_decision['No']/8])
print(entropy_weak)
print(entropy_weak.sum())

Entropy of Outcome based on weak wind
{'Yes': 6, 'No': 2}
[0.31127812 0.5       ]
0.8112781244591328


In [5]:
# Entroby for Outcome based on strong wind
print('Entropy of Outcome based on strong wind')
strong_decision = dict(tennis[tennis['Wind'] == 'Strong']['Decision'].value_counts())
print(strong_decision)
entropy_strong = entropy([strong_decision['Yes']/6, strong_decision['No']/6])
print(entropy_strong)
print(entropy_strong.sum())

Entropy of Outcome based on strong wind
{'No': 3, 'Yes': 3}
[0.5 0.5]
1.0


## Information Gain
**Information** Gain measures how much information a feature gives us about the decision (class). This is the main measurement used by a Decision Tree algorithm to construct a Decision Tree!

Decision Trees will always try to maximize information gain
The higher the information gain a feature has, the more likely it is to be tested first
the feature with the highest information gain will be the first feature in the decision tree, and its branches will lead to the other features

### Activity: Get the information gain for the wind column

In [6]:
0.94 - (8/14*entropy_weak.sum() + 6/14*entropy_strong.sum())

0.0478410717376383

### Acticity: Write a function to get the information gain of a given dataframe and its columns

In [7]:
def info_gain(df, y, features=[]):
    if len(features) == 0:
        features = df.drop(y, axis=1).columns
    
    gain = []
    entropy_y = df[y].unique()
    for index, value in enumerate(entropy_y):
        entropy_y[index] = len(df[df[y] == value])/len(df[y])
    entropy_y = entropy(entropy_y)
        
    for feature in features:
        entropy_feature = df[feature].unique()
        for index, value in enumerate(entropy_feature):
            unique_values = df[df[feature] == value][y].value_counts()
            values_dict = dict(unique_values)
            entropy_value = entropy([values_dict[unique_value]/unique_values.sum() for unique_value in unique_values.keys()]).sum()
            entropy_feature[index] = len(df[df[feature] == value][y])/len(df[feature])*entropy_value
        gain.append(entropy_y.sum() -  entropy_feature.sum())
    print(gain)
    
info_gain(tennis, 'Decision', ['Outlook', 'Temperature', 'Humidity', 'Wind'])

[0.24674981977443933, 0.02922256565895487, 0.15183550136234159, 0.04812703040826949]


## Build the decision tree with sklearn for tennis dataset
**For Decision Tree Visualization in Python:**

Packages that are needed are below. Note that the multiple installs for graphviz are to ensure the executables install correctly to avoid this error:

- conda install -c anaconda graphviz
- brew install graphviz
- conda install -c anaconda pydot
- conda install -c conda-forge pydotplus

In [8]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.tree import export_graphviz
import pydotplus

# read in the tennis data, need the extra parameters since it's a txt file
data = pd.read_csv('../Data/Tennis.csv')
print(data)

# encode the data so we can use it with our decision tree,
# by converting categories into numbers
data_encoded = data.apply(preprocessing.LabelEncoder().fit_transform)
print(data_encoded)

# create our decision tree classifier with entropy
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)

# one_hot_data = pd.get_dummies(data[['a', 'b', 'c', 'd']], drop_first=True)
# print(one_hot_data)

# provide our feature array and target array (1-item),
# and train the model using a decision tree
clf.fit(data_encoded[['Outlook', 'Temperature', 'Humidity', 'Wind']], data_encoded['Decision'])

# export our decision tree into data that can be plotted
dot_data = export_graphviz(clf, out_file=None, feature_names=['Outlook', 'Temp.', 'Humidity', 'Wind'])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('tennis_tree.png')

    Day   Outlook Temperature Humidity    Wind Decision
0     1     Sunny         Hot     High    Weak       No
1     2     Sunny         Hot     High  Strong       No
2     3  Overcast         Hot     High    Weak      Yes
3     4      Rain        Mild     High    Weak      Yes
4     5      Rain        Cool   Normal    Weak      Yes
5     6      Rain        Cool   Normal  Strong       No
6     7  Overcast        Cool   Normal  Strong      Yes
7     8     Sunny        Mild     High    Weak       No
8     9     Sunny        Cool   Normal    Weak      Yes
9    10      Rain        Mild   Normal    Weak      Yes
10   11     Sunny        Mild   Normal  Strong      Yes
11   12  Overcast        Mild     High  Strong      Yes
12   13  Overcast         Hot   Normal    Weak      Yes
13   14      Rain        Mild     High  Strong       No
    Day  Outlook  Temperature  Humidity  Wind  Decision
0     0        2            1         0     1         0
1     1        2            1         0     0   

True